In [ ]:
import os #operating system, file opens etc.
import tensorflow as tf
import numpy as np 
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential # to code a sequential or a funcional model. Sequential is easier to code
from keras.layers import Dense
import cv2 #computer vision package used for handling image files
import csv
import pandas as pd

Data available at: https://www.kaggle.com/competitions/datathonindoml-2022/data and was dowloaded to the drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
#!unzip -u "/content/gdrive/MyDrive/datathon/datathonindoml-2022.zip" -d "/content/gdrive/MyDrive/datathon/datathonindoml" For unzip used 1st time only


Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv(F"/content/gdrive/MyDrive/datathon/datathonindoml/train_labels.csv")

H = 200
W = 200
Size = 16000 #number of images used in training


X = np.zeros((Size,  H, W, 2), dtype="float32")
Y = np.zeros(Size, dtype="uint8") #label


In [ ]:
from tensorflow.keras.utils import to_categorical
img_f = np.zeros((H, W, 2))
for x in range(15984):
  print(x)
  imgid = int(df['id'][x])
  label1 = int(df['label'][x])
  #label2 = to_categorical(label1, 16)
  #print(label1)
  #print(label2)
  #print(label2.shape)
  #print(imgid)
  img = F"/content/gdrive/MyDrive/datathon/datathonindoml/train/train/"+str(imgid)+".tif"
  #print(img)

  img1=cv2.imread(img)
  #print(img1.shape)
  h = img1.shape[0]
  w = img1.shape[1]
  center0= int(h/2 - 100)
  center1 = int(w/2 - 100)
  #print(center0, center1) 
  img0_gray = img1[center0:center0+200,center1:center1+200, 0] #200x200
  img_f[:,:,[1]] = np.expand_dims(img0_gray/255, axis=2) #200x200x1, normalized data
  resized_img = cv2.resize(img1, (H, W), interpolation= cv2.INTER_CUBIC) #200x200x3
  img1_gray = resized_img[:,:,0] #200x200
  img_f[:,:,[0]] = np.expand_dims(img1_gray/255, axis=2) # 200x200x1
  #print(img_f.shape)
  X[x] = img_f
  Y[x] = label1
  #Y[x] = to_categorical(label1, 16)

print(X.shape)
print(Y.shape)


In [ ]:
###Define Model
#https://keras.io/guides/sequential_model/
#https://wiki.pathmind.com/convolutional-network
def get_model():
    #lrelu=tf.keras.layers.LeakyReLU(alpha=0.2)
    conv_args = {
        #"activation": "tanh",
        "activation": "relu",
        #"activation": lrelu,
        #"activation": None,
        "kernel_initializer": "Orthogonal", #method of weight initialization
        "padding": "same",
    }
    input0 = keras.Input(shape=(200, 200, 2)) #arrays called tensors in tensorflow/keras, shape is dimension

    x0 = layers.Conv2D(8, 3, **conv_args)(input0) #trials were done with 16 and 32 too
    x0 = layers.MaxPooling2D(pool_size=(2, 2))(x0) #pool layer 
    x0 = layers.Conv2D(8, 3, **conv_args)(x0)
    x0 = layers.MaxPooling2D(pool_size=(2, 2))(x0)
    x0 = layers.Conv2D(8, 3, **conv_args)(x0)
    x0 = layers.AveragePooling2D(pool_size=(2, 2))(x0)
    x0 = layers.Conv2D(1, 3, **conv_args)(x0)
    x0 = layers.Flatten()(x0)
    x0 = layers.Dense(256, activation='relu')(x0)
    x0 = layers.Dense(128, activation='relu')(x0)
    x0 = layers.Dense(16, activation='softmax')(x0)
    outputs = x0

    return keras.Model(input0, outputs)

model = get_model()
model.summary()
#loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_fn = keras.losses.CategoricalCrossentropy() # for classification models
model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])# metrics is for print only
#model.fit(X, Y, epochs=10, batch_size=1, callbacks=[cp_callback])
#model.fit(X, Y, epochs=10, batch_size=1, validation_data=(Z, W))
history = model.fit(X, to_categorical(Y, 16), epochs=10, batch_size=20)
#model.summary()
model.save(F"/content/gdrive/MyDrive/datathon/data_model_ms")
history = model.fit(X, to_categorical(Y, 16), epochs=10, batch_size=20)
#model.summary()
model.save(F"/content/gdrive/MyDrive/datathon/data_model_ms1")
history = model.fit(X, to_categorical(Y, 16), epochs=10, batch_size=20)
#model.summary()
model.save(F"/content/gdrive/MyDrive/datathon/data_model_ms2")


In [ ]:
f = open(F"/content/gdrive/MyDrive/datathon/out_ms2.csv", 'w')
writer = csv.writer(f)
data = ['id','label']
writer.writerow(data)
for x in range(16000):
  #prediction = model.predict(X) We want for single image
  #prediction = model.predict(X[x]) Model expects 4 dimension 200x200x2
  prediction = model.predict(np.expand_dims(X[x], axis=0)) #1x200x200x2
  pred1 = np.argmax(prediction)
  data = [x,pred1]
  writer.writerow(data)
f.close()

In [ ]:
df = pd.read_csv(F"/content/gdrive/MyDrive/datathon/filelist.csv")
H = 200
W = 200
Size = 900

X = np.zeros((Size,  H, W, 2), dtype="float32")
Y = np.zeros(Size, dtype="uint8")

from tensorflow.keras.utils import to_categorical
img_f = np.zeros((H, W, 2))
for x in range(900):
  print(x)
  imgid = int(df['id'][x])
  print(imgid)
  img = F"/content/gdrive/MyDrive/datathon/datathonindoml/validation/validation/"+str(imgid)+".tif"

  img1=cv2.imread(img)
  #print(img1.shape)
  h = img1.shape[0]
  w = img1.shape[1]
  center0= int(h/2 - 100)
  center1 = int(w/2 - 100)
  #print(center0, center1)
  img0_gray = cv2.cvtColor(img1[center0:center0+200,center1:center1+200], cv2.COLOR_BGR2GRAY)
  img_f[:,:,[1]] = np.expand_dims(img0_gray/255, axis=2)
  resized_img = cv2.resize(img1, (H, W), interpolation= cv2.INTER_CUBIC)
  img1_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
  img_f[:,:,[0]] = np.expand_dims(img1_gray/255, axis=2)
  print(img_f.shape)
  X[x] = img_f

print(X.shape)
print(Y.shape)


In [ ]:
f = open(F"/content/gdrive/MyDrive/datathon/out.csv", 'w')
writer = csv.writer(f)
data = ['id','label']
writer.writerow(data)
for x in range(16000):
  prediction = model.predict(np.expand_dims(X[x], axis=0))
  pred1 = np.argmax(prediction)
  data = [x,pred1]
  writer.writerow(data)
f.close()